In [120]:
pacman::p_load(readr, purrr, rlist, stringr, data.table, dplyr)

In [121]:
input_str <- "....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."

In [122]:
input_str <- read_file("puzzle_input.txt")

In [123]:
ncols <- input_str %>% str_split("\n") %>% .[[1]] %>% nchar %>% unique

input_mat <- matrix(input_str %>% str_replace_all("\n", "") %>% str_split("") %>% .[[1]], ncol = ncols, byrow = TRUE)

In [124]:
start_pos <- which(input_mat == '^', arr.ind=TRUE)

In [125]:
hashtags_pos <- which(input_mat == '#', arr.ind=TRUE) %>% as.data.table

# Part 1

In [126]:
find_next_location <- function(pos, hashtags_pos, direction) {
  row_current <- pos[1]
  col_current <- pos[2]
  visited[row_current, col_current] <<- TRUE
  
  if (direction == 'up') {
    next_pos <- as.matrix(hashtags_pos[col == col_current & row < row_current][order(-row)][1])
    next_pos[1] <- next_pos[1] + 1
    if(is.na(next_pos[1])){
      visited[1 : row_current, col_current] <<- TRUE
    } else{
      visited[next_pos[1] : row_current, col_current] <<- TRUE
    }
  } else if (direction == 'down') {
    next_pos <- as.matrix(hashtags_pos[col == col_current & row > row_current][order(row)][1])
    next_pos[1] <- next_pos[1] - 1
    if(is.na(next_pos[1])){
      visited[row_current:nrow(visited), col_current] <<- TRUE
    } else{
      visited[row_current:next_pos[1], col_current] <<- TRUE
    }
  } else if (direction == 'left') {
    next_pos <- as.matrix(hashtags_pos[col < col_current & row == row_current][order(-col)][1])
    next_pos[2] <- next_pos[2] + 1
    if(is.na(next_pos[1])){
      visited[row_current, 1:col_current] <<- TRUE
    } else{
      visited[row_current, next_pos[2]:col_current] <<- TRUE
    }
  } else if (direction == 'right') {
    next_pos <- as.matrix(hashtags_pos[col > col_current & row == row_current][order(col)][1])
    next_pos[2] <- next_pos[2] - 1
    if(is.na(next_pos[1])){
      visited[row_current, col_current:ncol(visited)] <<- TRUE
    } else{
      visited[row_current, col_current:next_pos[2]] <<- TRUE
    }
  } else{
    next_pos <- data.table(row = c(1), col = c(1)) %>% .[row > 1] %>% as.matrix
  }


  return(next_pos)  
}

In [127]:
specify_next_direction <- function(current_direction){
    switch(current_direction,
           'up' = 'right',
           'left' = 'up',
           'down' = 'left',
           'right' = 'down')
}

In [128]:
calculate_distance_traversed <- function(start_pos, next_pos, direction){
    row_start <- start_pos[1]
    col_start <- start_pos[2]
    
    row_next <- next_pos[1]
    col_next <- next_pos[2]
    
    if (direction %in% c('up', 'down')) {
        distance_traversed <- abs(row_next - row_start)
    } else {
        distance_traversed <- abs(col_next - col_start)
    }
    
    return(distance_traversed)
}

In [129]:
current_direction <- 'up'
in_map <- TRUE
current_pos <- start_pos
steps <- 0
visited <<- rep(FALSE, length(input_mat)) %>% matrix(nrow = nrow(input_mat), ncol = ncol(input_mat))

while(in_map){
    next_pos <- find_next_location(current_pos, hashtags_pos, current_direction)
    
    if(is.na(next_pos[1])){
        print("EXIT MAP")
        in_map <- FALSE
        break
    }
    distance_traversed <- calculate_distance_traversed(current_pos, next_pos, current_direction)
    current_direction <- specify_next_direction(current_direction)
    current_pos <- next_pos
    steps <- steps + distance_traversed
}

print(paste("Part 1:", sum(visited)))

[1] "EXIT MAP"
[1] "Part 1: 4647"


# Part 2

In [130]:
hashtags_pos <- which(input_mat == '#', arr.ind=TRUE) %>% as.data.table
num_adjenct_dt <- hashtags_pos %>% pmap(function(...){
    current_row <- list(...) %>% .[[1]]
    current_col <- list(...) %>% .[[2]]

    adjenct_row_ct <- hashtags_pos[abs(row - current_row) == 1] %>% nrow()
    adjenct_col_ct <- hashtags_pos[abs(col - current_col) == 1] %>% nrow()
    data.table(row = current_row, col = current_col, num_adjacent = adjenct_col_ct + adjenct_row_ct)
}) %>% rbindlist()

first_obstacle_loc <- find_next_location(start_pos, hashtags_pos, 'up')

num_adjenct_dt[row == first_obstacle_loc[1] - 1 & col == first_obstacle_loc[2], num_adjacent := num_adjacent + 1]

num_adjenct_dt[, ':='(IsOdd = num_adjacent %% 2 == 1)] %>% .[, sum(IsOdd)] * 2

[1] 786